In [1]:
import pandas as pd
import numpy as np
import re
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ===================== 2. 工具函数 =====================
def parse_date(date_str):
    try:
        if pd.isna(date_str) or date_str == "":
            return pd.NaT
        s = str(date_str).strip()
        m = re.search(r"\d{4}-\d{2}-\d{2}( \d{2}:\d{2}:\d{2})?", s)
        if m:
            s = m.group(0)
        return pd.to_datetime(s, errors="coerce")
    except Exception:
        return pd.NaT


def parse_int(val):
    try:
        if pd.isna(val):
            return 0
        s = str(val).replace(",", "").replace("，", "")
        s = re.sub(r"[^\d.-]", "", s)
        return int(float(s)) if s else 0
    except Exception:
        return 0


def parse_float(val):
    try:
        if pd.isna(val):
            return 0.0
        s = str(val).replace(",", "").replace("，", "")
        s = re.sub(r"[^\d.-]", "", s)
        return float(s) if s else 0.0
    except Exception:
        return 0.0


def parse_percentage(val):
    try:
        if pd.isna(val):
            return 0.0
        s = str(val).replace("%", "").replace("％", "")
        s = re.sub(r"[^\d.-]", "", s)
        return float(s) / 100 if s else 0.0
    except Exception:
        return 0.0






In [2]:
# ===================== 1. 读取原始数据 =====================
path_list = ["novel-1.csv", "novel-2.csv", "novel-3.csv","novel-4.csv"]
dfs = []
for path in path_list:
    if not os.path.exists(path):
        print(f"⚠️ 文件不存在: {path}")
        continue
    df_temp = pd.read_csv(path, encoding="utf-8-sig")
    dfs.append(df_temp)   # 列表 append 不使用关键字参数

# 合并为一个 DataFrame
if dfs:
    df = pd.concat(dfs, ignore_index=True)
else:
    df = pd.DataFrame()
print(f"已读取并合并 {len(dfs)} 个文件，最终行数：{len(df)}")

已读取并合并 4 个文件，最终行数：88593


In [3]:
#去掉版权转化这一列为“无匹配结果”的行
if "版权转化" in df.columns:
    before = len(df)
    df = df[df["版权转化"] != "无匹配结果"].reset_index(drop=True)
    df = df[df["评分"] != "0"].reset_index(drop=True)
    after = len(df)
    print(f"已删除 {before - after} 行（版权转化 == '无匹配结果'），剩余 {after} 行")
else:
    print("⚠️ 未找到列 '版权转化'，未执行删除操作")

已删除 3589 行（版权转化 == '无匹配结果'），剩余 85004 行


In [4]:
#检查缺失值
# 检查每一列是否有缺失值，如果有缺失值，删除掉缺失值所在行，并打印缺失值的数量
for col in df.columns:
    missing_count = df[col].isna().sum()
    if missing_count > 0:
        print(f"列 '{col}' 有 {missing_count} 个缺失值，正在删除对应行...")
        df = df[df[col].notna()].reset_index(drop=True)
print("缺失值处理完成。")

列 '作品视角' 有 352 个缺失值，正在删除对应行...
列 '所属系列' 有 436 个缺失值，正在删除对应行...
缺失值处理完成。


In [5]:
if "发表时间" in df.columns:
    df["发表时间"] = df["发表时间"].apply(parse_date)
    print("✅ 发表时间处理完成")

# ---------- 类型 one-hot ----------
if "类型" in df.columns:
    all_types = set()
    for x in df["类型"].dropna():
        all_types.update([t.strip() for t in str(x).split("-") if t.strip()])
    print(f"共有 {len(all_types)} 种【类型】")

    type_dict = {}
    for t in all_types:
        safe = re.sub(r'[^\w]', '_', t)
        type_dict[f"类型_{safe}"] = df["类型"].apply(lambda x: 1 if pd.notna(x) and t in str(x) else 0)

    if type_dict:
        df = pd.concat([df, pd.DataFrame(type_dict, index=df.index)], axis=1)
    print("✅ 类型 one-hot 完成")

# ---------- 作品视角 one-hot ----------
if "作品视角" in df.columns:
    view_df = pd.get_dummies(df["作品视角"].fillna("未知"), prefix="视角")
    df = pd.concat([df, view_df], axis=1)
    print("✅ 作品视角 one-hot 完成")

# ---------- 内容标签 one-hot ----------
if "内容标签" in df.columns:
    # 统一分隔并收集所有标签
    tag_lists = df["内容标签"].fillna("").apply(lambda s: [t.strip() for t in str(s).replace('，', ',').split(',') if t.strip() and t.strip() != "无"])
    all_tags = sorted({tag for lst in tag_lists for tag in lst})
    print(f"共有 {len(all_tags)} 种标签")

    tag_dict = {}
    for tag in all_tags:
        safe = re.sub(r'[^\w]', '_', tag)
        tag_dict[f"标签_{safe}"] = tag_lists.apply(lambda lst: 1 if tag in lst else 0)

    if tag_dict:
        df = pd.concat([df, pd.DataFrame(tag_dict, index=df.index)], axis=1)
    print("✅ 内容标签 one-hot 完成")

# ---------- 版权转化 ----------
if "版权转化" in df.columns:
    # 标记是否有版权；同时批量生成具体版权one-hot
    banquan_lists = df["版权转化"].fillna("").apply(lambda s: [t.strip() for t in str(s).replace('，', ',').split(',') if t.strip() and t.strip() != "未签约"])
    all_banquan = sorted({bq for lst in banquan_lists for bq in lst})
    banquan_dict = {}
    for bq in all_banquan:
        safe = re.sub(r'[^\w]', '_', bq)
        banquan_dict[f"版权_{safe}"] = banquan_lists.apply(lambda lst: 1 if bq in lst else 0)

    if banquan_dict:
        df = pd.concat([df, pd.DataFrame(banquan_dict, index=df.index)], axis=1)

    df["有版权转化"] = df["版权转化"].apply(lambda x: 0 if pd.isna(x) or str(x).strip() == "未签约" or str(x).strip()=="" else 1)
    print("✅ 版权转化标记完成（1=有版权，0=未签约）")

# ---------- 签约状态 ----------
if "签约状态" in df.columns:
    df["签约状态_二分类"] = df["签约状态"].apply(
        lambda x: 0 if pd.notna(x) and str(x).strip() == "未签约" else 1
    )
    print("✅ 签约状态 二分类完成")

# ---------- 数值字段 ----------
num_cols = ["字数", "非v章节章均点击数", "总书评数", "当前被收藏数", "文章积分", "评价人数"]
for c in num_cols:
    if c in df.columns:
        df[c] = df[c].apply(parse_int)
print("✅ 数值列清洗完成")

# ---------- 评分 ----------
if "评分" in df.columns:
    df["评分"] = df["评分"].replace("暂无", np.nan).apply(parse_float)
    print("✅ 评分列处理完成")

# ---------- 百分比字段 ----------
pct_cols = ["五星比例", "四星比例", "三星比例", "二星比例", "一星比例"]
for c in pct_cols:
    if c in df.columns:
        df[c] = df[c].apply(parse_percentage)
print("✅ 百分比列处理完成")

# ---------- 输出结果 ----------
out_path = "novel_processed.csv"
df.to_csv(out_path, index=False, encoding="utf-8-sig")
print(f"\n🎯 数据处理完成，已保存到：{out_path}")
print(f"共 {len(df)} 条记录，{len(df.columns)} 个字段\n")
print("=== 数值列摘要 ===")
print(df.select_dtypes(include=["int64", "float64"]).describe())

✅ 发表时间处理完成
共有 37 种【类型】
✅ 类型 one-hot 完成
✅ 作品视角 one-hot 完成
共有 277 种标签
✅ 内容标签 one-hot 完成
✅ 版权转化标记完成（1=有版权，0=未签约）
✅ 签约状态 二分类完成
✅ 数值列清洗完成
✅ 评分列处理完成
✅ 百分比列处理完成

🎯 数据处理完成，已保存到：novel_processed.csv
共 84216 条记录，378 个字段

=== 数值列摘要 ===
                 字数     非v章节章均点击数          总书评数        当前被收藏数          文章积分  \
count  8.421600e+04  8.421600e+04  8.421600e+04  8.421600e+04  8.421600e+04   
mean   3.032073e+05  3.244344e+04  9.198989e+03  2.178953e+04  3.668109e+08   
std    2.736010e+05  1.524142e+05  8.416203e+04  1.212035e+05  2.183915e+09   
min    2.993400e+04  0.000000e+00  0.000000e+00  0.000000e+00  2.245700e+04   
25%    1.337838e+05  2.570000e+02  9.000000e+00  5.000000e+01  3.281241e+06   
50%    2.422160e+05  1.719000e+03  1.370000e+02  9.450000e+02  1.780318e+07   
75%    3.863110e+05  9.934500e+03  9.542500e+02  5.781000e+03  7.912757e+07   
max    6.622435e+06  3.182059e+06  3.694838e+06  3.131198e+06  7.553632e+10   

                 评分          评价人数          五星比例          四星比例   

In [6]:
# 删除作者、名称、发表时间数值完全相同的行（保留首次出现）
cols = ['作者', '名称', '发表时间']
missing = [c for c in cols if c not in df.columns]
if missing:
    print(f"⚠️ 无法去重，缺少列: {missing}")
else:
    # 规范化发表时间（转换为 datetime 并按秒四舍五入）以避免微小差异导致无法匹配
    df['__发表时间_norm'] = pd.to_datetime(df['发表时间'], errors='coerce').dt.round('s')
    before = len(df)
    df = df.drop_duplicates(subset=['作者', '名称', '__发表时间_norm'], keep='first').reset_index(drop=True)
    df = df.drop(columns='__发表时间_norm')
    after = len(df)
    print(f"去重完成：已删除 {before - after} 行，剩余 {after} 行")


去重完成：已删除 2525 行，剩余 81691 行


In [7]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81691 entries, 0 to 81690
Columns: 378 entries, 作者 to 签约状态_二分类
dtypes: bool(9), datetime64[ns](1), float64(6), int64(353), object(9)
memory usage: 230.7+ MB


,字数,发表时间,非v章节章均点击数,总书评数,当前被收藏数,文章积分,评分,评价人数,五星比例,四星比例,...,版权_网络剧签约,版权_网络电影签约,版权_舞台剧签约,版权_衍生品签约,版权_非大陆电影签约,版权_非大陆电视剧签约,版权_非大陆网络剧签约,版权_音乐签约,有版权转化,签约状态_二分类
count,8.169100e+04,81691,8.169100e+04,8.169100e+04,8.169100e+04,8.169100e+04,81691.000000,81691.000000,81691.000000,81691.000000,...,81691.000000,81691.000000,81691.000000,81691.000000,81691.000000,81691.000000,81691.000000,81691.000000,81691.000000,81691.000000
mean,2.950463e+05,2023-05-18 20:37:12.940911360,1.752797e+04,3.847358e+03,1.090067e+04,1.784489e+08,6.188710,74.004652,0.377386,0.244846,...,0.005447,0.000233,0.000282,0.002350,0.000024,0.000037,0.000049,0.000220,0.036577,0.736336
min,2.993400e+04,2005-06-23 20:40:46,0.000000e+00,0.000000e+00,0.000000e+00,2.245700e+04,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.303965e+05,2022-08-17 10:11:28,2.470000e+02,8.000000e+00,4.500000e+01,3.074956e+06,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.366990e+05,2023-05-20 18:00:00,1.548000e+03,1.220000e+02,8.370000e+02,1.643374e+07,8.300000,3.000000,0.330000,0.170000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,3.772470e+05,2024-03-12 14:46:40,8.402000e+03,7.930000e+02,4.911000e+03,6.790507e+07,9.200000,20.000000,0.670000,0.450000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,6.622435e+06,2025-11-16 23:45:30,3.182059e+06,3.694838e+06,3.131198e+06,7.553632e+10,10.000000,47765.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
std,2.640206e+05,NaN,7.609805e+04,4.008388e+04,5.555438e+04,1.038741e+09,4.100644,658.684956,0.363514,0.281856,...,0.073605,0.015249,0.016777,0.048423,0.004948,0.006060,0.006997,0.014842,0.187722,0.440622


In [8]:
#查看版权转化的情况，版权转化列的标签为“版权_转化类型”，只统计其中为1的个数
try:
    copyright_cols = [c for c in df.columns if c.startswith("版权_")]
    if not copyright_cols:
        print("⚠️ 未找到以 '版权_' 开头的列。请确认已经执行过 one-hot 生成步骤或列名前缀是否一致。")
    else:
        counts = df[copyright_cols].sum().sort_values(ascending=False)
        print("✅ 各版权类型被标记为1的数量（降序）：")
        print(counts.to_string())
        # 若存在汇总标记列，显示其统计
        if "有版权转化" in df.columns:
            total_with_bq = int(df["有版权转化"].astype(bool).sum())
            print(f"\n📌 有版权转化 总数: {total_with_bq} / {len(df)}（记录数）")
        # 输出前10并在 notebook 中展示表格
        top10 = counts.head(10).reset_index()
        top10.columns = ["版权类型", "数量"]
        from IPython.display import display
        display(top10)
except Exception as e:
    print("统计版权转化时报错：", e)

✅ 各版权类型被标记为1的数量（降序）：
版权_中国大陆出版最新签约     1960
版权_广播剧签约           870
版权_繁体出版_港_台_签约     722
版权_亚洲出版签约          634
版权_网络剧签约           445
版权_有声读物签约          354
版权_电视剧签约           303
版权_影视签约            293
版权_衍生品签约           192
版权_漫画签约            180
版权_游戏签约            109
版权_动漫签约             98
版权_欧美出版签约           88
版权_港澳出版签约           76
版权_电影签约             36
版权_动态漫画签约           30
版权_动画签约             25
版权_广播剧线下活动签约        24
版权_舞台剧签约            23
版权_网络电影签约           19
版权_音乐签约             18
版权_外文广播剧签约          13
版权_漫画线下活动签约          9
版权_微短剧签约             6
版权_主题店签约             6
版权_品牌联名签约            5
版权_短视频签约             5
版权_非大陆网络剧签约          4
版权_非大陆电视剧签约          3
版权_短剧签约              2
版权_非大陆电影签约           2

📌 有版权转化 总数: 2988 / 81691（记录数）


,版权类型,数量
0,版权_中国大陆出版最新签约,1960
1,版权_广播剧签约,870
2,版权_繁体出版_港_台_签约,722
3,版权_亚洲出版签约,634
4,版权_网络剧签约,445
5,版权_有声读物签约,354
6,版权_电视剧签约,303
7,版权_影视签约,293
8,版权_衍生品签约,192
9,版权_漫画签约,180
